# 05-2 교차 검증과 그리드 서치
### 검증 세트가 필요한 이유와 교차 검증 학습 및 그리드 서치와 랜덤 서치를 이용해 최적의 성능을 내는 하이퍼파라미터 탐색
---
</br>

💡용어 정리💡  
❓ 검증 세트 ➡️ 테스트 세트를 사용하지 않고, 과대/과소적합을 판단하기 위해 훈련 세트를 또 나누는 것  
 --> 기존 >> 20%(테스트 세트) + 80%(훈련 세트)  
 --> 검증 세트 추가 >> 20%(테스트 세트) + 20%(검증 세트) + 60%(훈련 세트)  
 1. 훈련 세트에서 모델 훈련, 검증 세트에서 모델 평가  
 2. 1 반복 수행 후, 가장 좋은 모델 셀렉  
 3. 훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델 재훈련  
 4. 테스트 세트에서 최종 점수 평가  
 
❓ 교차 검증 ➡️ 훈련 세트를 다시 훈련 세트와 검증 세트로 분할하는 작업을 반복한 뒤, 점수들의 평균을 내는 것 --> k-폴드 교차 검증  
❓ 하이퍼파라미터 ➡️ 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터 <-> 모델 파라미터  
❓ 그리드 서치 ➡️ GridSearchCV 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행함!  
 (별도의 cross_validate() 함수 호출 필요xx)  

In [3]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [4]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

**훈련 세트와 테스트 세트로 분할**

In [5]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

**훈련 세트를 다시 훈련 세트와 검증 세트로 분할!!**

In [6]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

**DecisionTreeClassifier 이용하여 모델 훈련**

In [7]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)

dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))
# 과대적합 --> 매개변수를 바꿔서 더 좋은 모델 탐색 필요!

0.9971133028626413
0.864423076923077


**cross_validate()를 이용한 5-폴드 교차 검증**
- cross_validate() --> fit_time, score_time, test_score 3개의 키가 각각 5개의 숫자를 포함  

In [8]:
from sklearn.model_selection import cross_validate

# 기본적으로 5-폴드 교차 검증 수행!
# (평가할 모델 객체, 훈련 세트의 특성, 훈련 세트의 타깃) 
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.03969145, 0.03184676, 0.03915262, 0.04552054, 0.03819609]), 'score_time': array([0.00320864, 0.00105739, 0.00112009, 0.00794816, 0.00122142]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


**평균 구하기**
 --> scores 리스트 중 test_score에 담긴 값들의 평균 구하기

In [9]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


**그리드 서치 - GridSearchCV 클래스**

In [10]:
from sklearn.model_selection import GridSearchCV # 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행!

# 탐색 하고자 하는 매개변수와 탐색할 값의 리스트를 딕셔너리로 생성!
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [11]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) # n_jobs=-1 --> CPU의 모든 코어 사용

In [14]:
# 모델 훈련
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [17]:
# 그리드 서치는 훈련이 끝난 25개의 모델 중 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델 훈련!
# best_estimater_ --> 검증 점수가 가장 높은 모델이 저장되는 속성
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [19]:
# best_params_ --> 최적의 매개변수 저장
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [21]:
# cv_results_['mean_test_score'] --> 각 매개변수에서 수행한 교차 검증의 평균 점수가 저장되는 키
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


**좀 더 복잡한 매개변수 조합 탐색! --> 매개변수 간의 간격을 0.0001 혹은 1로 설정하여**

In [23]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001), # 0.0001 부터 0.0001씩 증가하여 0.001이 되는 배열 생성
          'max_depth' : range(5, 20, 1), # 5부터 1씩 증가하여 20까지
          'min_samples_split' : range(2, 100, 10) # 2부터 10씩 증가하여 100까지
          }

In [24]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [25]:
print(gs.best_params_) # 최상의 매개변수 조합 확인

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [26]:
print(np.max(gs.cv_results_['mean_test_score '])) # 최상의 교차 검증 점수 확인

0.8683865773302731


**좀 더 복잡한 매개변수 조합 탐색! --> 매개변수 간의 간격을 미리 정하지 않고!**
- 랜덤 서치 사용! --> 매개변수 값의 목록이 아니고, 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달  


In [27]:
from scipy.stats import uniform, randint # 확률 분포 클래스 임포트, 각각 실숫값과 정숫값을 뽑음 == 난수 발생기

**randint 사용법**

In [29]:
# 0~10 범위를 갖는 randint 객체 생성
rgen = randint(0, 10)

# 10개의 숫자 샘플링
rgen.rvs(10)

array([5, 7, 1, 0, 1, 3, 3, 4, 6, 2])

In [30]:
np.unique(rgen.rvs(1000), return_counts=True) # 0~10까지 1000개 샘플링 후, 각 숫자의 개수 확인

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 91, 107, 107, 102, 110,  92,  92, 101,  99,  99]))

**uniform 사용법**

In [31]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.15768807, 0.54659179, 0.84242934, 0.63338613, 0.24607475,
       0.3898265 , 0.09315282, 0.79225939, 0.46073752, 0.77246676])

### 최적의 매개변수 조합 찾기!
**탐색할 매개변수의 딕셔너리 생성 --> params**

In [34]:
params = {'min_impurity_decrease' : uniform(0.0001, 0.001), # 0.0001~0.001 사이의 실숫값 샘플링
          'max_depth' : randint(20, 50), # 20~50 사이의 정숫값 샘플링
          'min_samples_split' : randint(2, 25), # 2~25 사이의 정숫값 샘플링
          'min_samples_leaf' : randint(1, 25), # 리프 노드가 되기 위한 최소 샘플의 개수
          }

**모델 훈련**

In [35]:
from sklearn.model_selection import RandomizedSearchCV
# 위에서 정의한 params에 정의된 매개변수 범위에서 총 100번(n_iter=100) 샘플링하여 교차 검증 수행 --> 최적의 매개변수 조합 찾음!!
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f88e6ce9e90>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f88e6cf0e50>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f88e6ce9bd0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f88e6ce9750>},
                   random_state=42)

**최적의 매개변수 조합 출력**

In [36]:
print(gs.best_params_) # 최적의 매개변수 조합 출력

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


**최고의 교차 검증 점수**

In [37]:
print(np.max(gs.cv_results_['mean_test_score'])) # 최고의 교차 검증 점수

0.8695428296438884


**위 모델을 최종 모델로 결정 --> 테스트 세트로 성능 확인!**

In [39]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
